In [1]:
import json
import types
import requests

In [2]:
class Namespace:
    def __init__(self, adict):
        self.__dict__.update(adict)

In [3]:
base_link = "http://localhost:8080/api/v1/"
#base_link = "http://192.168.0.218:8080/api/v1/"

In [4]:
links = {
    "position":Namespace({
        "all":"position/all",
        "all_active":"position/all/active",
        "single":"position/"
    }),
    "employee":Namespace({
        "all":"employee/all",
        "single":"employee/"
    }),
    "task":Namespace({
        "all":"task/all",
        "single":"task/"
    }),
    "taskrole":Namespace({
        "all":"taskrole/all",
        "single":"taskrole/",
        "position":"taskrole/position/"#{p}
    }),
    "logbook":Namespace({
        "all":"logbook/all",
        "range":"logbook/range/", #{s}/{e}
        "prange":"logbook/range/",#{p}/{s}/{e}
        "single":"logbook/"
    }),
    "absence":Namespace({
        "all":"absence/all",
        "prange":"absence/range/",#{p}
        "single":"absence/"
    }),
    "evaluation":Namespace({
        "all":"evaluation/all",
        "single":"evaluation/",
        "position":"evaluation/position/",#{p}
    }),
    "course":Namespace({
        "all":"course/all",
        "single":"course/",
        "position":"course/position/",#{p}
    })
};links_ns = Namespace(links)

In [5]:
def get_direction_name(p):
    p = Namespace(p)
    if p.service is not None:
        subdiv = p.service["subdivision"]
        subdiv = Namespace(subdiv)
        if subdiv.direction is not None:
            return subdiv.direction["shortName"]
        else: return subdiv.shortName
    else:
        div = p.subdivision["direction"]
        if div is not None:
            return div["shortName"]
        else: return p.subdivision["shortName"]
        
def get_service_name(p):
    p = Namespace(p)
    if p.service is not None:
        return p.service["name"]
    else:
        return "AUCUN"

In [6]:
def get_employee_bio_data(p):
    emp_data_list = list()
    positions = [requests.get(base_link+links_ns.position.single+str(p)).json()]
    for p in positions:
        emp_data = dict()
        emp_data["position_id"] = p["positionId"]
        emp_data["employee_id"] = p["employee"]["employeeId"]
        emp_data["fullname"] = p["employee"]["lastName"] +" "+ p["employee"]["firstName"].strip() 
        emp_data["matricula"] = p["employee"]["matricule"]
        emp_data["rolename"] = p["role"]["name"]
        emp_data["service"] = get_service_name(p)
        emp_data["direction"] = get_direction_name(p)
        emp_data["email"] = p["employee"]["email"]
        emp_data["contact"] = p["employee"]["contact"]
        emp_data["location"] = p["location"]
        emp_data_list.append(emp_data)
    return emp_data_list[0]

In [7]:
requests.get(base_link+links_ns.position.all_active).url

'http://localhost:8080/api/v1/position/all/active'

In [8]:
get_employee_bio_data(5)

{'position_id': 5,
 'employee_id': 5,
 'fullname': 'BOKOUA Sébastien',
 'matricula': '1281021B',
 'rolename': 'SOUS-DIRECTEUR',
 'service': 'AUCUN',
 'direction': 'DSESP',
 'email': 'sebastien.bokoua@arti.ci',
 'contact': '',
 'location': 'ABIDJAN'}

In [9]:
from datetime import datetime, timedelta

def time_diff(start_time_str, end_time_str="08:10:00"):
    start_time = datetime.strptime(start_time_str, '%H:%M:%S')
    end_time = datetime.strptime(end_time_str, '%H:%M:%S')
    time_delta = end_time - start_time
    time_diff_seconds = int(time_delta.total_seconds())
    return time_diff_seconds

In [15]:
log_data = requests.get(base_link+links_ns.logbook.prange+'2023-03-23/2023-03-23').json()

In [16]:
from itertools import groupby

In [17]:
grouped_data = []
for key, group in groupby(log_data, lambda x: x['position']['positionId']):
    # Create a list of "logbookId" for each "positionId"
    #grouped_data[key] = [x['logbookId'] for x in group]
    for x in group:
        checkIn = x['checkinTime']
        checkOut = x['checkoutTime']
        d = {
        "logbookId":x['logbookId'],
        "positionId":x['position']['positionId'],
        "employeeId":x['position']["employee"]["employeeId"],
        "fullName":x['position']["employee"]["lastName"]+' '+x['position']["employee"]["firstName"],
        "checkIn":checkIn,
        "checkOut":checkOut,
        "logCount":2,
        "isLate": time_diff(checkIn) < 0
    }
    grouped_data.append(d)
# Print the result
print(grouped_data)

[{'logbookId': 880, 'positionId': 1, 'employeeId': 1, 'fullName': 'ANGOUA Guy-Charles', 'checkIn': '00:00:00', 'checkOut': '00:00:00', 'logCount': 2, 'isLate': False}, {'logbookId': 901, 'positionId': 2, 'employeeId': 2, 'fullName': 'ANOH Affanou Lisette', 'checkIn': '00:00:00', 'checkOut': '00:00:00', 'logCount': 2, 'isLate': False}, {'logbookId': 922, 'positionId': 3, 'employeeId': 3, 'fullName': 'BAILLY Eric Théodore', 'checkIn': '09:40:00', 'checkOut': '20:30:00', 'logCount': 2, 'isLate': True}, {'logbookId': 943, 'positionId': 4, 'employeeId': 4, 'fullName': 'BEN Mourad', 'checkIn': '08:03:00', 'checkOut': '18:05:00', 'logCount': 2, 'isLate': False}, {'logbookId': 964, 'positionId': 5, 'employeeId': 5, 'fullName': 'BOKOUA Sébastien', 'checkIn': '00:00:00', 'checkOut': '00:00:00', 'logCount': 2, 'isLate': False}, {'logbookId': 985, 'positionId': 6, 'employeeId': 6, 'fullName': 'CISSE Alassane', 'checkIn': '08:14:00', 'checkOut': '17:32:00', 'logCount': 2, 'isLate': True}, {'logbook

In [19]:
presence = len (grouped_data)
lateCount = len([1 for x in grouped_data if x["isLate"]==True])
logDate = log_data[0]["logDate"]
meta_data = {
    "presence":presence,
    "lateCount":lateCount,
    "logDate":logDate
}
meta_data

{'presence': 27, 'lateCount': 5, 'logDate': '2023-03-23'}

# Late Count

In [53]:
from pandas import json_normalize

In [54]:
log_data = requests.get(base_link+links_ns.logbook.prange+'2023-03-01/2023-03-31').json()

In [57]:
compute_workdays("2023-03-01")

-1443

In [44]:
df = json_normalize(log_data)

In [45]:
#df.columns

In [46]:
#df.head()

In [47]:
df["isLate"] = df["checkinTime"].map(time_diff) < 0

In [48]:
df_sn = df[["logbookId","isLate","position.positionId"]]
df_sn = df_sn.groupby('position.positionId').agg({'isLate': 'sum', 'logbookId': 'count'}).reset_index()

In [49]:
df_sn

,position.positionId,isLate,logbookId
0,1,3,21
1,2,14,21
2,3,5,21
3,4,1,21
4,5,1,21
5,6,11,21
6,7,0,21
7,8,3,21
8,10,0,21
9,11,4,21


In [ ]:
cols = ["position.positionId","position.employee.lastName","position.employee.firstName","isLate_x","logbookId_x"]
cols_new = ["positionId","lastName","fisrtName","lateCount","presenceCount"]

In [ ]:
df_ns = df_sn.merge(df, on='position.positionId', how="left")[cols].drop_duplicates()

In [ ]:
df_ns.columns = cols_new

In [ ]:
df_ns

,positionId,lastName,fisrtName,lateCount,presenceCount
0,1,ANGOUA,YOBOUATH GUY CHARLES,0,1
1,2,ANOH,AMA LISETTE DESIREE Epse AFFANOU,2,4
5,3,BAILLY,ERIC THEODORE,1,3
8,4,BEN,MOURAD MOHAMED ALY,2,4
12,5,BOKOUA,ZIOGBA SEBASTIEN,2,3
15,6,CISSE,ALASSANE AL MOUSTAPHA,0,1
16,7,COULIBALY,MOUSSA,1,2
18,8,DEBEY,ALLEGUY LINDA CAROLE,0,1
19,10,DJIRE,KELETIGUI LAMINE,0,1
20,11,ENAN,EBLOTCHE JEAN-JACQUES,0,1
